# Intro a Django Forms

In [ ]:
Form: una forma de recolectar data de los usuarios

GET - Obtener data como usuarios
POST - Guardar data como usuarios

In [ ]:
python manage.py startapp forms_test

In [ ]:
src/forms_test/forms.py

In [ ]:
from django import forms

class SearchForm(forms.Form):
    q = forms.CharField()

## Mostrar el form en una vista

In [ ]:
src/forms_test/views.py

In [ ]:
from django.shortcuts import render

from .forms import SearchForm

def home(request):
    form = SearchForm()
    return render(request, 'forms.html', { 'form' : form })

In [ ]:
src/templates/forms.html

In [ ]:
<form method="POST" action=".">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="Guardar">
    {% if delete_url %}
        <a href="{{ delete_url }}">Eliminar</a>
    {% endif %}
</form>

In [ ]:
src/config/urls.py

In [ ]:
#...
urlpatterns = [
    path("up/", include("up.urls")),
    path("", include("pages.urls")),
    path("analytics/", include("analytics.urls")),
    path("admin/", admin.site.urls),
    path("forms/", include("forms_test.urls")),#<-----
]
#...

In [ ]:
src/forms_test/urls.py

In [ ]:
from django.contrib import admin
from django.urls import path

from .views import home

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", home),
]

In [ ]:
src/config/settings.py

In [ ]:
#...
INSTALLED_APPS = [
    "forms_test.apps.FormsTestConfig",#<-----
    "accounts.apps.AccountsConfig",
    "addresses.apps.AddressesConfig",
    "analytics.apps.AnalyticsConfig",
    "billing.apps.BillingConfig",
    "carts.apps.CartsConfig",
    "order.apps.OrderConfig",
    "pages.apps.PagesConfig",
    "products.apps.ProductsConfig",
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]
#...

## Validacion en campos de un Form

In [ ]:
src/forms_test/forms.py

In [ ]:
from django import forms

class TestForm(forms.Form):
    some_text = forms.CharField()
    boolean = forms.BooleanField()
    integer = forms.IntegerField()
    email = forms.EmailField()

In [ ]:
src/forms_test/views.py

In [ ]:
from django.shortcuts import render

from .forms import TestForm

def home(request):
    form = TestForm()
    return render(request, 'forms.html', { 'form' : form })

## Obtener data de un Django form

In [ ]:
src/forms_test/views.py

In [ ]:
from django.shortcuts import render

from .forms import TestForm

def home(request):
    form = TestForm(request.POST or None)
    if form.is_valid():
        print(form.cleaned_data)
    return render(request, 'forms.html', { 'form' : form })

## Validaciones basicas en un form

In [ ]:
src/forms_test/forms.py

In [ ]:
from django import forms

class TestForm(forms.Form):
    un_texto = forms.CharField()
    boolean = forms.BooleanField()
    entero = forms.IntegerField()
    email = forms.EmailField()

    def clean_entero(self, *args, **kwargs):
        entero = self.clean_data.get('entero')
        if entero > 100:
            raise forms.ValidationError('El numero debe ser menor o igual que 100')
        return entero
    
    def clean_un_texto(sefl, *args, **kwargs):
        un_texto = sefl.cleaned_data.get('un_texto')
        if len(un_texto) < 10:
            raise forms.ValidationError('El texto debe contener mas de 10 caracteres')

## Data inicial en un form

### Opcion 1

In [ ]:
src/forms_test/views.py

In [ ]:
from django.shortcuts import render

from .forms import TestForm

def home(request):
    initial_data = {
        'un_texto': 'Texto inicial',
        'boolean': True,
        'entero': 100,
        'email': 'test@test.com'
    }
    form = TestForm(request.POST or None, initial=initial_data)
    if form.is_valid():
        print(form.cleaned_data)
    return render(request, 'forms.html', { 'form' : form })

### Opcion 2

In [ ]:
src/forms_test/forms.py

In [ ]:
from django import forms

class TestForm(forms.Form):
    un_texto = forms.CharField(initial='Texto inicial')
    boolean = forms.BooleanField()
    entero = forms.IntegerField(initial=100)
    email = forms.EmailField()

    def clean_entero(self, *args, **kwargs):
        entero = self.cleaned_data.get('entero')
        if entero > 100:
            raise forms.ValidationError('El numero debe ser menor o igual que 100')
        return entero
    
    def clean_un_texto(sefl, *args, **kwargs):
        un_texto = sefl.cleaned_data.get('un_texto')
        if len(un_texto) < 10:
            raise forms.ValidationError('El texto debe contener mas de 10 caracteres')

## Etiqueta y Widgets de Forms

In [ ]:
src/forms_test/forms.py

In [ ]:
from django import forms

MY_CHOICES = {
    ('db-value1', 'Opcion 1'),
    ('o2', 'Opcion 2'),
    ('o3', 'Opcion 3')
}

YEARS = [x for x in range(1900, 2030)]


class TestForm(forms.Form):
    fecha = forms.DateField(widget=forms.SelectDateWidget(years=YEARS))
    un_texto = forms.CharField(initial='Texto inicial', label='Ingresa un texto:', widget=forms.Textarea(attrs={'rows':4, 'cols':20}))
    boolean = forms.BooleanField(initial=True)
    entero = forms.IntegerField(initial=80)
    email = forms.EmailField(initial='test@test.com')
    opciones = forms.CharField(label='Selecciona una opcion', widget=forms.Select(choices=MY_CHOICES))
    opciones_radio = forms.CharField(label='Selecciona una opcion', widget=forms.RadioSelect(choices=MY_CHOICES))
    opciones_checkbox = forms.CharField(label='Selecciona una opcion', widget=forms.CheckboxSelectMultiple(choices=MY_CHOICES))

    def clean_entero(self, *args, **kwargs):
        entero = self.cleaned_data.get('entero')
        if entero > 100:
            raise forms.ValidationError('El numero debe ser menor o igual que 100')
        return entero
    
    def clean_un_texto(sefl, *args, **kwargs):
        un_texto = sefl.cleaned_data.get('un_texto')
        if len(un_texto) < 10:
            raise forms.ValidationError('El texto debe contener mas de 10 caracteres')

## Model form

In [ ]:
src/templates/forms.html

In [ ]:
{% extends 'base.html' %}
{% block content %}
<h1>Form View</h1>
<form method="POST" action=".">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="Guardar">
    {% if delete_url %}
        <a href="{{ delete_url }}">Eliminar</a>
    {% endif %}
</form>
{% endblock %}

In [ ]:
src/forms_test/forms.py

In [ ]:
#...

from .models import Product

class ProductModelForm(forms.ModelForm):
    class Meta:
        model = Product
        fields = [
            'title',
            'slug',
            'price'
        ]
        exclude = []
#...

In [ ]:
src/forms_test/models.py

In [ ]:
from django.conf import settings
from django.db import models

class Product(models.Model):
    title = models.CharField(max_length=120)
    slug = models.SlugField(unique=True)
    price = models.DecimalField()

In [ ]:
python manage.py makemigrations
python manage.py migrate 

In [ ]:
src/products/admin.py

In [ ]:
from django.contrib import admin

from .models import Product

admin.site.register(Product)

## Mensajes de error personalizados

In [ ]:
src/forms_test/forms.py

In [ ]:
#...
class ProductModelForm(forms.ModelForm):
    labels = {
        'title': 'Mi etiqueta para el titulo',
        'slug': 'Mi etiqueta para el slug',
        'proce': 'Mi etiqueta para el precio'
    }
    class Meta:
        model = Product
        fields = [
            'title',
            'slug',
            'price'
        ]
        exclude = []

    def clean_title(self, *args, **kwargs):
        title = self.cleaned_data.get('title')
        if len(title) <= 10:
            raise forms.ValidationError('El titulo debe tener mas de 10 caracteres')
        return title
    
    def clean_slug(self, *args, **kwargs):
        slug = self.cleaned_data.get('slug')
        if len(slug) <= 10:
            raise forms.ValidationError('El slug debe tener mas de 10 caracteres')
        if 'misupermarca' not in slug:
            raise forms.ValidationError('El slug debe incluir "misupermarca"')
        return slug
#...

## Django formsets

In [ ]:
src/templates/base.html

In [ ]:
<body>
    <div class="container">
        {% block content %}
        {% endblock %}
    </div>
</body>

In [ ]:
src/templates/formset_view.html

In [ ]:
{% extends 'base.html' %}
{% block content %}
<h1>Form View</h1>
<form method="POST" action=".">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="Guardar">
    {% if delete_url %}
        <a href="{{ delete_url }}">Eliminar</a>
    {% endif %}
</form>
{% endblock %}

In [ ]:
src/forms_test/views.py

In [ ]:
from django.forms import formset_factory, modelform_factory
from django.shortcuts import render

from .forms import TestForm, ProductModelForm

def home(request):
    TestFormSet = formset_factory(TestForm, extra=3)
    formset = TestFormSet(request.POST or None)
    if formset.is_valid():
        for form in formset:
            print(form.cleaned_data)
    context = {
        'formset': formset
    }
    return render(request, 'formset_view.html', context)

## Django Modelfomrsets

MoelFormset: Un grupo de ModelForms

In [ ]:
src/forms_test/views.py

In [ ]:
from django.forms import formset_factory, modelform_factory
from django.shortcuts import render

from .forms import TestForm, ProductModelForm
from .models import Product

def home(request):
    ProductModelFormSet = modelform_factory(Product, form==ProductModelForm)
    formset = ProductModelFormSet(request.POST or None, queryset=Product.objects.all())
    print('formset.data')
    print(formset.data)

    print('formset.errors')
    print(formset.errors)

    formset.clean()
    if formset.is_valid():
        print('ModelFormset es valido')
        
    context = {
        'formset': formset
    }
    return render(request, 'formset_view.html', context)

## Registro de usuario

In [ ]:
src/forms_test/models.py

In [ ]:
from django.conf import settings
from django.db import models
from django.contrib.auth.models import AbstractBaseUser

class Product(models.Model):
    title = models.CharField(max_length=120)
    slug = models.SlugField(unique=True)
    price = models.DecimalField(decimal_places=2, max_digits=20)

class User(AbstractBaseUser):
    email = models.EmailField(max_length=255, unique=True)
    full_name = models.CharField(max_length=255, blank=True, null=True)
    is_active = models.BooleanField(default=True)
    staff = models.BooleanField(default=False)
    admin = models.BooleanField(default=False)
    timestamp = models.DateTimeField(auto_now_add=True)

    USERNAME_FIELD = "email"
    REQUIRED_FIELDS = []

In [ ]:
src/forms_test/forms.py

In [ ]:
#...
class UserModelForm(forms.ModelForm):
    class Meta:
        model = User
        fields = [
            'email',
            'full_name'
        ]
        exclude = ['is_active', 'staff', 'admin', 'timestamp']   
#...

In [ ]:
src/forms_test/views.py

In [ ]:
from django.shortcuts import render

from .forms import UserModelForm

def home(request):
    form = UserModelForm(request.POST or None)
    if form.is_valid():
        form.save()
    return render(request, 'forms.html', { 'form' : form })